In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
# Start with loading all necessary libraries
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/vietnamese-online-news-csv-dataset/After.csv")

In [ ]:
df['topic'] = df['topic'].str.strip()

In [ ]:
def topic_clean(df, target, num):
    freq = df[df["source"] == target]['topic'].value_counts()
    # Set a flag to indicate if a value should be turned into NaN
    should_be_none = False

    # Iterate over the values in the column
    for value in freq.index:
        # If the value only appears once, set the flag to True
        if (freq[value] <= num) or (freq[value] == np.nan):
            should_be_none = True
    
        # If the flag is True, turn the value into NaN
        if should_be_none:
            df.loc[df[(df['source'] == target) & (df['topic'] == value)].index, 'topic'] = np.nan
    
        # Reset the flag
        should_be_none = False

    # Replace all None values with nan
    df = df.replace("None", np.nan)

In [ ]:
source_list = df['source'].unique()[:-1].tolist()

In [ ]:
threshold = {'laodong': 20, 
        'zingnews': 20, 
        'thanhnien.vn': 0,
        '24h.com.vn' : 20,
        'vtv.vn' : 36,
        'vnexpress' : 0,
        'danviet': 61,
        'dantri': 61,
        'vov.vn': 0,
        'tienphong': 3,
        'tuoitre.vn':7,
        'soha': 10,
        'nld': 19,
        'kenh14': 83,
        'vietnamnet.vn': 50,
        'docbao.vn': 0,
        'cafebiz': 3,
        'vtc.vn': 100,
        'bongdaplus': 20,
        'baoquocte': 25,
        'anninhthudo': 10,
        'eva.vn': 76,
        'thethaovanhoa': 2,
        'baochinhphu': 60,
        'qdnd.vn': 2
       }

In [ ]:
for i in tqdm(range(len(source_list))):
    target = source_list[i]
    topic_clean(df, target, threshold[target])

df = df.replace("None", np.nan)

In [ ]:
df.to_csv('After_remove_label.csv')

===============================================================================================

In [ ]:
org = pd.read_csv("/kaggle/input/vietnamese-online-news-csv-dataset/Original.csv")
tar = pd.read_csv("/kaggle/input/vietnamese-online-news-csv-dataset/Target.csv")

In [ ]:
def group_label(df, target, org, tar):
    temp = df[df["source"] == target]
    temp_org = org[target].dropna().tolist()
    temp_tar = tar[target].dropna().tolist()
    for i in range(len(temp_org)):
        if len(temp_org) != len(temp_tar):
            print('not matching length in:', target)
            break
        df.loc[df[(df['source'] == target) & (df['topic'] == temp_org[i].strip())].index, 'topic'] = temp_tar[i].strip()

In [ ]:
for i in tqdm(range(len(source_list))):
    target = source_list[i]
    group_label(df, target, org, tar)

In [ ]:
df['topic'].unique()

In [ ]:
df.loc[df[(df['source'].isnull()) & (df['topic'] == "['Thế giới', 'Quân sự']")].index, 'topic'] = 'Thế giới'
df.loc[df[(df['source'].isnull()) & (df['topic'] == "Kinh Doanh")].index, 'topic'] = 'Kinh doanh - Tài chính'

In [ ]:
df.to_csv('After_group_label.csv')

==============================================================================================

In [ ]:
df['title'].fillna('', inplace=True)
df['content'].fillna('', inplace=True)

In [ ]:
df['title'] = df['title'].str.replace('\d+', '')
df['content'] = df['content'].str.replace('\d+', '')

In [ ]:
def convert_float_to_string(text):
    try:
        return str(text)
    except TypeError:
        return text

df["content"] = df["content"].apply(convert_float_to_string)
df["title"] = df["title"].apply(convert_float_to_string)

In [ ]:
import re

# Import the stopwords from the .txt file
with open("/kaggle/input/vietnamese-news-finals/vnesestopwords.txt", "r") as f:
    stopwords = f.read().splitlines()

# Remove the stopwords from the dataframe
def remove_stopwords(text):
    if type(text) == float:
        return text
    else:
        words = text.split(" ")
        new_words = []
        for word in words:
            if word not in stopwords:
                new_words.append(word)
        return " ".join(new_words)

df["content"] = df["content"].apply(remove_stopwords)
df["title"] = df["title"].apply(remove_stopwords)

In [ ]:
df.to_csv('temp.csv')

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Apply the function to the 'text' column
df['content'] = df['content'].apply(remove_punctuation)
df['title'] = df['title'].apply(remove_punctuation)

In [ ]:
df['content'] = df['content'].str.lower()
df['title'] = df['title'].str.lower()

In [ ]:
df.to_csv('temp.csv')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [ ]:
df.shape

In [ ]:
labeled_data = df[df['topic'].notnull()]
unlabeled_data = df[df['topic'].isnull()]

In [ ]:
unlabeled_data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    labeled_data[['title', 'content']],
    labeled_data['topic'],
    test_size=0.3,
    random_state=42
)

In [ ]:
y_train.value_counts()

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train_vectorized = vectorizer.fit_transform(X_train['title'] + ' ' + X_train['content'])

In [ ]:
X_train_vectorized = X_train_vectorized.toarray()
X_train_vectorized.shape

In [ ]:
classifier = SVC(kernel = 'linear')

In [ ]:
classifier.fit(X_train_vectorized, y_train)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB

# nb_classifier = MultinomialNB()

In [ ]:
# nb_classifier.fit(X_train_vectorized, y_train)

In [ ]:
X_test_vectorized = vectorizer.transform(X_test['title'] + ' ' + X_test['content'])
accuracy = classifier.score(X_test_vectorized, y_test)
print(f"Accuracy: {accuracy}")

In [ ]:
# X_test_vectorized = vectorizer.transform(X_test['title'] + ' ' + X_test['content']).toarray()
# accuracy = nb_classifier.score(X_test_vectorized, y_test)
# print(f"Accuracy: {accuracy}")

In [ ]:
unlabeled_data_vectorized = vectorizer.transform(unlabeled_data['title'] + ' ' + unlabeled_data['content'])

In [ ]:
predicted_topics = nb_classifier.predict(unlabeled_data_vectorized)

In [ ]:
predicted_topics.shape

In [ ]:
unlabeled_data.loc[:, 'topic'] = predicted_topics

In [ ]:
merged_df = df.merge(unlabeled_data, on='id', how='left')
merged_df['topic_x'] = merged_df['topic_x'].fillna(merged_df['topic_y'])
df = merged_df.rename(columns={'topic_x': 'topic',
                                  'Unnamed: 0_x': 'Unnamed: 0',
                                  'author_x': 'author',
                                  'content_x': 'content',
                                  'picture_count_x': 'picture_count',
                                  'processed_x': 'processed',
                                  'source_x': 'source',
                                  'title_x': 'title',
                                  'crawled_at_x': 'crawled_at',
                                  'url_x': 'url'})

In [ ]:
df[df['topic'].isnull()].shape

In [ ]:
df = df.drop(df.columns[[11,12,13,14,15,16,17,18,19,20]], axis=1)

In [ ]:
df.to_csv('The_last.csv')

In [ ]:
print('All done.')